In [25]:
#Load Dependencies
import pandas as pd
import os
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func
import pymysql
from config import db_pass

import warnings
warnings.filterwarnings('ignore')

Extract CSV into DataFrames 

In [26]:
#Read in df csv
input_file = os.path.join('Resources','companylist_nasdq.csv')
df_ndaq = pd.read_csv(input_file)

# Display the Data
df_ndaq

df_ndaq = df_ndaq.rename(columns={"Summary Quote": "SummaryQuote"})

df_ndaq.head()

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,SummaryQuote,Unnamed: 8
0,YI,"111, Inc.",8.650,$705.36M,2018.0,Health Care,Medical/Nursing Services,https://www.nasdaq.com/symbol/yi,NaN
1,PIH,"1347 Property Insurance Holdings, Inc.",5.065,$30.45M,2014.0,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pih,NaN
2,PIHPP,"1347 Property Insurance Holdings, Inc.",24.911,$17.44M,NaN,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pihpp,NaN
3,TURN,180 Degree Capital Corp.,1.940,$60.38M,NaN,Finance,Finance/Investors Services,https://www.nasdaq.com/symbol/turn,NaN
4,FLWS,"1-800 FLOWERS.COM, Inc.",19.410,$1.25B,1999.0,Consumer Services,Other Specialty Stores,https://www.nasdaq.com/symbol/flws,NaN


Transforming the DataFrame


In [27]:
cleandf_ndaq = df_ndaq[["Symbol", "Name", "LastSale", "MarketCap",
                       "IPOyear", "Sector", "industry", "SummaryQuote"]]
cleandf_ndaq.head()

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,SummaryQuote
0,YI,"111, Inc.",8.650,$705.36M,2018.0,Health Care,Medical/Nursing Services,https://www.nasdaq.com/symbol/yi
1,PIH,"1347 Property Insurance Holdings, Inc.",5.065,$30.45M,2014.0,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pih
2,PIHPP,"1347 Property Insurance Holdings, Inc.",24.911,$17.44M,NaN,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pihpp
3,TURN,180 Degree Capital Corp.,1.940,$60.38M,NaN,Finance,Finance/Investors Services,https://www.nasdaq.com/symbol/turn
4,FLWS,"1-800 FLOWERS.COM, Inc.",19.410,$1.25B,1999.0,Consumer Services,Other Specialty Stores,https://www.nasdaq.com/symbol/flws


In [28]:
# Identify incomplete rows
cleandf_ndaq.count()

Symbol          3439
Name            3439
LastSale        3437
MarketCap       3171
IPOyear         1600
Sector          2970
industry        2970
SummaryQuote    3439
dtype: int64

In [29]:
# Drop all rows with missing information
cleandf_ndaq = cleandf_ndaq.dropna(how='any')

In [30]:
# Verify dropped rows
cleandf_ndaq.count()

Symbol          1398
Name            1398
LastSale        1398
MarketCap       1398
IPOyear         1398
Sector          1398
industry        1398
SummaryQuote    1398
dtype: int64

In [31]:
# Identifying datatype
cleandf_ndaq.dtypes

Symbol           object
Name             object
LastSale        float64
MarketCap        object
IPOyear         float64
Sector           object
industry         object
SummaryQuote     object
dtype: object

In [32]:
cleandf_ndaq["MarketCapNum"]= pd.DataFrame(cleandf_ndaq["MarketCap"].str.slice(1,-1))
cleandf_ndaq["MarketCapType"]= pd.DataFrame(cleandf_ndaq['MarketCap'].str.slice(-1))
cleandf_ndaq['MarketCapNum'] = pd.to_numeric(cleandf_ndaq['MarketCapNum'])

In [33]:
cleandf_ndaq["MarketCapNum1"] = cleandf_ndaq.apply(lambda row: row.MarketCapNum * 1000 if row.MarketCapType == 'B' else row.MarketCapNum , axis = 1) 
cleandf_ndaq

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,SummaryQuote,MarketCapNum,MarketCapType,MarketCapNum1
0,YI,"111, Inc.",8.6500,$705.36M,2018.0,Health Care,Medical/Nursing Services,https://www.nasdaq.com/symbol/yi,705.36,M,705.36
1,PIH,"1347 Property Insurance Holdings, Inc.",5.0650,$30.45M,2014.0,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pih,30.45,M,30.45
4,FLWS,"1-800 FLOWERS.COM, Inc.",19.4100,$1.25B,1999.0,Consumer Services,Other Specialty Stores,https://www.nasdaq.com/symbol/flws,1.25,B,1250.00
5,BCOW,"1895 Bancorp of Wisconsin, Inc.",9.6500,$47.06M,2019.0,Finance,Banks,https://www.nasdaq.com/symbol/bcow,47.06,M,47.06
8,VNET,"21Vianet Group, Inc.",8.0500,$904.76M,2011.0,Technology,"Computer Software: Programming, Data Processing",https://www.nasdaq.com/symbol/vnet,904.76,M,904.76
9,TWOU,"2U, Inc.",61.6100,$3.6B,2014.0,Technology,Computer Software: Prepackaged Software,https://www.nasdaq.com/symbol/twou,3.60,B,3600.00
10,QFIN,"360 Finance, Inc.",20.0200,$2.88B,2018.0,Finance,Finance: Consumer Services,https://www.nasdaq.com/symbol/qfin,2.88,B,2880.00
11,JOBS,"51job, Inc.",93.8700,$5.81B,2004.0,Technology,Diversified Commercial Services,https://www.nasdaq.com/symbol/jobs,5.81,B,5810.00
12,JFKKU,8i Enterprises Acquisition Corp,10.0700,$50.35M,2019.0,Finance,Business Services,https://www.nasdaq.com/symbol/jfkku,50.35,M,50.35
17,ABIL,Ability Inc.,1.4200,$11.42M,2014.0,Capital Goods,Telecommunications Equipment,https://www.nasdaq.com/symbol/abil,11.42,M,11.42


In [35]:
ndaq_load_cols = ["Symbol", "Name", "LastSale", "MarketCapNum1", "IPOyear", "Sector", "industry", "SummaryQuote"]
ndaq_load= cleandf_ndaq[ndaq_load_cols].copy()
ndaq_load = ndaq_load.rename(columns={"MarketCapNum1": "MarketCap"})
ndaq_load.head()


,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,SummaryQuote
0,YI,"111, Inc.",8.650,705.36,2018.0,Health Care,Medical/Nursing Services,https://www.nasdaq.com/symbol/yi
1,PIH,"1347 Property Insurance Holdings, Inc.",5.065,30.45,2014.0,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pih
4,FLWS,"1-800 FLOWERS.COM, Inc.",19.410,1250.00,1999.0,Consumer Services,Other Specialty Stores,https://www.nasdaq.com/symbol/flws
5,BCOW,"1895 Bancorp of Wisconsin, Inc.",9.650,47.06,2019.0,Finance,Banks,https://www.nasdaq.com/symbol/bcow
8,VNET,"21Vianet Group, Inc.",8.050,904.76,2011.0,Technology,"Computer Software: Programming, Data Processing",https://www.nasdaq.com/symbol/vnet


Create Database Connection

In [36]:
connection_string = f"root:{db_pass}@localhost/stocks_db"
engine = create_engine(f'mysql+pymysql://{connection_string}')
    

In [37]:
engine.table_names()

['all_stocks_5yr',
 'cleandf_ndaq',
 'companylist_nasdq',
 'companylist_nyse',
 'industry_analysis']

Load DataFrame into Database


In [44]:
#Load Dataframe into database
#Confirm load
ndaq_load.to_sql(name='companylist_nasdq', con=engine, if_exists='append', index=False)

In [46]:
#Confirm load
pd.read_sql_query('select * from companylist_nasdq', con=engine).head()

,id,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,SummaryQuote
0,1,YI,"111, Inc.",8.650,705.36,2018,Health Care,Medical/Nursing Services,https://www.nasdaq.com/symbol/yi
1,2,PIH,"1347 Property Insurance Holdings, Inc.",5.065,30.45,2014,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pih
2,3,FLWS,"1-800 FLOWERS.COM, Inc.",19.410,1250.00,1999,Consumer Services,Other Specialty Stores,https://www.nasdaq.com/symbol/flws
3,4,BCOW,"1895 Bancorp of Wisconsin, Inc.",9.650,47.06,2019,Finance,Banks,https://www.nasdaq.com/symbol/bcow
4,5,VNET,"21Vianet Group, Inc.",8.050,904.76,2011,Technology,"Computer Software: Programming, Data Processing",https://www.nasdaq.com/symbol/vnet
